# Visualize White Island Rainfall Data For Crater Lake Level Assessment

Rainfal data stored in a series of (monthly) CSV files scraped from Metconnect (using a separate notebook).
Lake level data from FITS.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

**rainfall data**

In [ ]:
files = sorted(glob.glob('*.csv'))

df = pd.DataFrame()
for f in files:
    month = pd.read_csv(f, index_col='Date', parse_dates=True)
    df = df.append(month)

**lake level data**

In [ ]:
url = 'https://fits.geonet.org.nz/observation?siteID=WI200&typeID=z&methodID=img&days=290'
ll = pd.read_csv(url, parse_dates=True, index_col='date-time')

In [ ]:
fig,(ax1,ax2) = plt.subplots(nrows=2,ncols=1, figsize=(15,10))
ax1.bar(df.index, df['Rainfall'])
ax1.set_ylabel('rainfall (mm)')
ax2.plot(df.index, np.cumsum(df['Rainfall']))
ax2.set_ylabel('cumulative rainfall (mm)')

ax3 = ax2.twinx()
ax3.plot(ll.index, ll[' z (m)'], marker='o', color='#ff7f0e')
ax3.set_ylim([-1,11])
ax3.tick_params(axis='y')
ax3.set_ylabel('lake level (m asl)', color='#ff7f0e')

plt.savefig('rainfall.png')

# Lake Catchment Analysis

Rainfall and Water Level Rise
--

During 2018, the lake has risen about 10 m.

During the same period, there has been about 1 m of direct rainfall on the lake. Allowing for ~2.5 month of missing data, I assume 1.3 m of rainfall on the lake.

Catchment Area
--

This is about 5.5 times the area of the lake. The GE image is for 2011, but the lake area won't have increased by more than 10% in the 2016 eruption???
![](lake_catchment_area.png)

Rainfall Accounts for About 70% of the Lake Level Rise in 2018
--

During 2018, about 1.3 m * 5.5 ~= 7 m of water ended up in the lake. Assuming no evaporation, etc. This is about 70% of the lake level rise in the same period.

If we assume the lake increased in area by 10% in the 2016 eruption, the area ratio is about 5. This implies about 1.3 m * 5 ~= 6.5 m of water ended up in the lake, about 65% of the lake level rise.